In [1]:
import numpy as np

# Load data
X_train = np.load('../data/spacenet/X_train.npy')
y_train = np.load('../data/spacenet/y_train.npy')
X_val = np.load('../data/spacenet/X_val.npy')
y_val = np.load('../data/spacenet/y_val.npy')

# Print shapes and stats
print("✅ Data loaded successfully!")
print(f"X_train shape: {X_train.shape} | dtype: {X_train.dtype} | range: [{X_train.min():.3f}, {X_train.max():.3f}]")
print(f"y_train shape: {y_train.shape} | dtype: {y_train.dtype} | unique: {np.unique(y_train)}")
print(f"X_val shape:   {X_val.shape}")
print(f"y_val shape:   {y_val.shape}")

✅ Data loaded successfully!
X_train shape: (996, 256, 256, 4) | dtype: float32 | range: [0.000, 1.000]
y_train shape: (996, 256, 256) | dtype: float32 | unique: [0. 1.]
X_val shape:   (249, 256, 256, 4)
y_val shape:   (249, 256, 256)


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model

# --- Custom Metrics (IoU & Dice) ---
def iou(y_true, y_pred, smooth=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # Threshold at 0.5
    y_true = tf.cast(y_true, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    union = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3]) - intersection
    return tf.reduce_mean((intersection + smooth) / (union + smooth))

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    denom = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    return tf.reduce_mean((2. * intersection + smooth) / (denom + smooth))

# --- Build Lightweight UNet ---
def build_unet(input_shape=(256, 256, 4)):
    inputs = Input(input_shape)
    
    # Encoder
    c1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(32, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D(2)(c1)
    
    c2 = Conv2D(64, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(64, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D(2)(c2)
    
    c3 = Conv2D(128, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(128, 3, activation='relu', padding='same')(c3)
    
    # Decoder
    u4 = UpSampling2D(2)(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(64, 3, activation='relu', padding='same')(u4)
    c4 = Conv2D(64, 3, activation='relu', padding='same')(c4)
    
    u5 = UpSampling2D(2)(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(32, 3, activation='relu', padding='same')(u5)
    c5 = Conv2D(32, 3, activation='relu', padding='same')(c5)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(c5)
    return Model(inputs, outputs)

# --- Build and Compile ---
print("Building UNet model...")
model = build_unet(input_shape=(256, 256, 4))
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', iou, dice_coef]
)

print("✅ Model built and compiled!")
print("Input shape:", model.input_shape)
print("Output shape:", model.output_shape)
print("Total parameters:", f"{model.count_params():,}")

Building UNet model...
✅ Model built and compiled!
Input shape: (None, 256, 256, 4)
Output shape: (None, 256, 256, 1)
Total parameters: 471,841


In [4]:
import numpy as np

# Load data
X_train = np.load('../data/spacenet/X_train.npy')
y_train = np.load('../data/spacenet/y_train.npy')
X_val = np.load('../data/spacenet/X_val.npy')
y_val = np.load('../data/spacenet/y_val.npy')

# ✅ FIX: Add channel dimension to masks
y_train = np.expand_dims(y_train, axis=-1)  # (996, 256, 256) → (996, 256, 256, 1)
y_val = np.expand_dims(y_val, axis=-1)      # (249, 256, 256) → (249, 256, 256, 1)

print("✅ Masks reshaped to include channel dimension!")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")

✅ Masks reshaped to include channel dimension!
y_train shape: (996, 256, 256, 1)
y_val shape: (249, 256, 256, 1)


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
import numpy as np
import os

# --- Load and reshape data (already done, but included for completeness) ---
X_train = np.load('../data/spacenet/X_train.npy')
y_train = np.expand_dims(np.load('../data/spacenet/y_train.npy'), axis=-1)
X_val = np.load('../data/spacenet/X_val.npy')
y_val = np.expand_dims(np.load('../data/spacenet/y_val.npy'), axis=-1)

# --- Custom Metrics ---
def iou(y_true, y_pred, smooth=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    union = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3]) - intersection
    return tf.reduce_mean((intersection + smooth) / (union + smooth))

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    denom = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    return tf.reduce_mean((2. * intersection + smooth) / (denom + smooth))

# --- Build Model ---
def build_unet(input_shape=(256, 256, 4)):
    inputs = Input(input_shape)
    c1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(32, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D(2)(c1)
    
    c2 = Conv2D(64, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(64, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D(2)(c2)
    
    c3 = Conv2D(128, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(128, 3, activation='relu', padding='same')(c3)
    
    u4 = UpSampling2D(2)(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(64, 3, activation='relu', padding='same')(u4)
    c4 = Conv2D(64, 3, activation='relu', padding='same')(c4)
    
    u5 = UpSampling2D(2)(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(32, 3, activation='relu', padding='same')(u5)
    c5 = Conv2D(32, 3, activation='relu', padding='same')(c5)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(c5)
    return Model(inputs, outputs)

# --- Compile ---
print("Building model...")
model = build_unet()
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', iou, dice_coef]
)

# --- Train ---
os.makedirs("../results", exist_ok=True)
print("🚀 Starting training...")
history = model.fit(
    X_train, y_train,
    batch_size=4,
    epochs=15,
    validation_data=(X_val, y_val),
    verbose=1
)

# --- Save ---
model.save("../models/unet_building_segmentation.h5")
print("✅ Model saved to ../models/unet_building_segmentation.h5")

Building model...
🚀 Starting training...
Epoch 1/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 328s 1s/step - accuracy: 0.9216 - dice_coef: 0.0016 - iou: 8.9281e-04 - loss: 0.2718 - val_accuracy: 0.9199 - val_dice_coef: 1.2227e-09 - val_iou: 1.2227e-09 - val_loss: 0.2630
Epoch 2/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.9254 - dice_coef: 1.6188e-09 - iou: 1.6188e-09 - loss: 0.2491 - val_accuracy: 0.9199 - val_dice_coef: 1.2227e-09 - val_iou: 1.2227e-09 - val_loss: 0.2564
Epoch 3/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 308s 1s/step - accuracy: 0.9254 - dice_coef: 1.6188e-09 - iou: 1.6188e-09 - loss: 0.2483 - val_accuracy: 0.9199 - val_dice_coef: 1.2227e-09 - val_iou: 1.2227e-09 - val_loss: 0.2758
Epoch 4/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 317s 1s/step - accuracy: 0.9257 - dice_coef: 0.0119 - iou: 0.0065 - loss: 0.2422 - val_accuracy: 0.9205 - val_dice_coef: 0.0178 - val_iou: 0.0091 - val_loss: 0.2441
Epoch 5/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 306s 1s/step - accuracy: 0.9259 - dice_coef: 0.0365 - i

✅ Model saved to ../models/unet_building_segmentation.h5


In [6]:
import matplotlib.pyplot as plt
import numpy as np

# Load model and data
from tensorflow.keras.models import load_model
model = load_model("../models/unet_building_segmentation.h5", custom_objects={'iou': iou, 'dice_coef': dice_coef})

X_val = np.load('../data/spacenet/X_val.npy')
y_val = np.expand_dims(np.load('../data/spacenet/y_val.npy'), axis=-1)

# Predict on first 3 validation samples
print("Generating sample predictions...")
preds = model.predict(X_val[:3])

# Plot
os.makedirs("../results", exist_ok=True)
plt.figure(figsize=(15, 5))
for i in range(3):
    # Original RGB
    plt.subplot(3, 3, i*3 + 1)
    plt.imshow(X_val[i][:,:,:3])
    plt.title(f"Input {i+1} (RGB)")
    plt.axis('off')
    
    # Ground truth
    plt.subplot(3, 3, i*3 + 2)
    plt.imshow(y_val[i].squeeze(), cmap='gray')
    plt.title("Ground Truth")
    plt.axis('off')
    
    # Prediction (thresholded)
    plt.subplot(3, 3, i*3 + 3)
    plt.imshow((preds[i].squeeze() > 0.5), cmap='gray')
    plt.title("Prediction")
    plt.axis('off')

plt.tight_layout()
plt.savefig("../results/segmentation_samples.png", dpi=150)
plt.close()
print("✅ Sample predictions saved to ../results/segmentation_samples.png")

Generating sample predictions...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 732ms/step
✅ Sample predictions saved to ../results/segmentation_samples.png


In [9]:
import numpy as np
from tensorflow.keras.models import load_model

# Load model (with custom metrics)
model = load_model(
    "../models/unet_building_segmentation.h5",
    custom_objects={'iou': iou, 'dice_coef': dice_coef}
)

# Load validation data
X_val = np.load('../data/spacenet/X_val.npy')
y_val = np.expand_dims(np.load('../data/spacenet/y_val.npy'), axis=-1)

# Evaluate
print("Evaluating on full validation set...")
metrics = model.evaluate(X_val, y_val, verbose=1)

# Extract results
loss, accuracy, iou_score, dice_score = metrics
print(f"\n FINAL TEST RESULTS:")
print(f"  IoU:        {iou_score:.4f}")
print(f"  Dice:       {dice_score:.4f}")
print(f"  Accuracy:   {accuracy:.4f}")
print(f"  Loss:       {loss:.4f}")

Evaluating on full validation set...
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.9240 - dice_coef: 0.1588 - iou: 0.0916 - loss: 0.2196

 FINAL TEST RESULTS:
  IoU:        0.0916
  Dice:       0.1588
  Accuracy:   0.9240
  Loss:       0.2196


In [10]:
# Save metrics to file
with open("../results/segmentation_evaluation.txt", "w") as f:
    f.write("Week 3: Image Segmentation Evaluation\n")
    f.write(f"IoU: {iou_score:.4f}\n")
    f.write(f"Dice: {dice_score:.4f}\n")
    f.write(f"Accuracy: {accuracy:.4f}\n")

print(" Evaluation saved to ../results/segmentation_evaluation.txt")

 Evaluation saved to ../results/segmentation_evaluation.txt


In [11]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model

# Load model and data
model = load_model("../models/unet_building_segmentation.h5", custom_objects={'iou': iou, 'dice_coef': dice_coef})
X_val = np.load('../data/spacenet/X_val.npy')
y_val = np.expand_dims(np.load('../data/spacenet/y_val.npy'), axis=-1)

# Select 4 diverse samples
indices = [10, 50, 100, 150]  # rural, suburban, urban, complex
plt.figure(figsize=(12, 10))
for i, idx in enumerate(indices):
    pred = model.predict(X_val[idx:idx+1])[0]
    pred_mask = (pred.squeeze() > 0.5).astype(np.uint8)
    
    # RGB Input
    plt.subplot(4, 3, i*3 + 1)
    plt.imshow(X_val[idx][:,:,:3])
    plt.title(f"Input {i+1}")
    plt.axis('off')
    
    # Ground Truth
    plt.subplot(4, 3, i*3 + 2)
    plt.imshow(y_val[idx].squeeze(), cmap='gray')
    plt.title("Ground Truth")
    plt.axis('off')
    
    # Prediction
    plt.subplot(4, 3, i*3 + 3)
    plt.imshow(pred_mask, cmap='gray')
    plt.title("Prediction")
    plt.axis('off')

plt.tight_layout()
plt.savefig("../results/segmentation_qualitative.png", dpi=200, bbox_inches='tight')
plt.close()
print("✅ Qualitative plot saved: segmentation_qualitative.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 743ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
✅ Qualitative plot saved: segmentation_qualitative.png


In [12]:
# Assuming you saved history earlier (if not, retrain briefly or skip)
# If you have history.history, use it. Otherwise, create a mock-up from your logs:

epochs = list(range(1, 16))
train_iou = [0.0009, 0.0000, 0.0000, 0.0065, 0.0205, 0.0103, 0.0295, 0.0474, 0.0652, 0.0637, 0.0714, 0.0756, 0.0862, 0.0840, 0.0836]
val_iou = [0.0000, 0.0000, 0.0000, 0.0091, 0.0002, 0.0323, 0.0583, 0.0487, 0.0736, 0.0799, 0.1004, 0.0924, 0.0397, 0.1080, 0.0914]

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_iou, 'b-o', label='Train IoU')
plt.plot(epochs, val_iou, 'r-o', label='Val IoU')
plt.title('IoU Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('IoU')
plt.legend()
plt.grid(True)

# Dice (approximate from IoU: Dice ≈ 2*IoU/(1+IoU))
train_dice = [2*i/(1+i) for i in train_iou]
val_dice = [2*i/(1+i) for i in val_iou]
plt.subplot(1, 2, 2)
plt.plot(epochs, train_dice, 'b-o', label='Train Dice')
plt.plot(epochs, val_dice, 'r-o', label='Val Dice')
plt.title('Dice Coefficient Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Dice')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig("../results/segmentation_metrics.png", dpi=150)
plt.close()
print("✅ Metrics plot saved: segmentation_metrics.png")

✅ Metrics plot saved: segmentation_metrics.png


In [13]:
# Final metrics from your evaluation
metrics = {
    'IoU': 0.0916,
    'Dice': 0.1588,
    'Accuracy': 0.9240
}

plt.figure(figsize=(6, 4))
bars = plt.bar(metrics.keys(), metrics.values(), color=['steelblue', 'crimson', 'seagreen'])
plt.title('Week 3: Segmentation Model Performance')
plt.ylabel('Score')
plt.ylim(0, 1)

# Add value labels on bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, f'{yval:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig("../results/segmentation_summary.png", dpi=150)
plt.close()
print("✅ Summary plot saved: segmentation_summary.png")

✅ Summary plot saved: segmentation_summary.png
